<a href="https://colab.research.google.com/github/adali981/metagenomics/blob/master/AQUACULTURE_BIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

path files ws: 


*   AGER Project:
        BMR: /home/zpl/storage/anto_aquaculture/Ager_bmr_analysis
        ANTO (Hermetia/Nanoplastics): /home/zpl/storage/AntoniaB_1128_16S/raw
        ANTO (Tenebrio/Tanzania): /home/zpl/storage/anto_aquaculture/190204_M00571_0367_000000000-C92CJ/Project_AntoniaB_871_16S_ZPL-FEM
*   BAZZOCCHI Project:
        /home/zpl/storage/Proj_1008_16S/raw_file_Pr1008/bazzocchi_raw
*   MYSUSHI Project:
        /home/zpl/storage/mysushi_bioinfo/152253

\\
\
path files drive:


*   AGER Project:
        Ager_bioinfo / Ager_BMR
*   BAZZOCCHI Project:
        Bazzocchi_fish
*   MYSUSHI Project:
        mysushi_bioinfo



SILVA 138 \
SILVA_138_SSURef_NR99_tax_silva.fasta file (510,985 sequence)

pwd: ~/storage/SILVA_138

In [0]:
## clean

awk '/^>/{print s? s"\n"$0:$0;s="";next}{s=s sprintf("%s",$0)}END{if(s)print s}' SILVA_138_SSURef_NR99_tax_silva.fasta > SILVA_138_SSURef_NR99_tax_silva_OL.fasta

grep -A 1 'Bacteria;' SILVA_138_SSURef_NR99_tax_silva_OL.fasta > Bacteria_SILVA_138_SSURef_NR99_tax_silva_OL.fasta

sed -i '/--/d' Bacteria_SILVA_138_SSURef_NR99_tax_silva_OL.fasta

grep 'Bacteria;' headers_SILVA_138_SSURef_NR99.txt > headers_bacteria_SILVA_138_SSURef_NR99.txt

# 432033

sed 's/>//' headers_bacteria_SILVA_138_SSURef_NR99.txt > Bacteria_tax_SILVA_138_SSURef_NR99.txt

sed 's/ Bacteria;/\tBacteria;/' Bacteria_tax_SILVA_138_SSURef_NR99.txt > Bacteria_tax_SILVA_138_SSURef_NR99_mod.txt

In [0]:
# import in qza

qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path Bacteria_SILVA_138_SSURef_NR99_tax_silva_OL.fasta \
  --output-path Bacteria_SILVA_138_SSURef_NR99.qza

qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path Bacteria_tax_SILVA_138_SSURef_NR99_mod.txt \
  --output-path Bacteria_taxonomy_SILVA_138_SSURef_NR99.qza

# AGER

## BMR

In [0]:
# import data

qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path Ager_bmr \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path ager_bmr.qza


# visualize (#10000 default value)

qiime demux summarize \
  --i-data ager_bmr.qza \
  --p-n 8000 \
  --o-visualization ager_bmr.qzv

Minimum:	31730 \
Median:	79942.0 \
Mean:	84228.375 \
Maximum:	159878 \
Total:	4042962

In [0]:
# remove primers

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences ager_bmr.qza \
    --p-cores 10 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GACTACHVGGGTATCTAATCC \
    --verbose \
    --p-discard-untrimmed \
    --o-trimmed-sequences ager_bmr_trimmed.qza


# visualize

qiime demux summarize \
  --i-data ager_bmr_trimmed.qza \
  --o-visualization ager_bmr_trimmed.qzv


# remove primers (b)

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences ager_bmr.qza \
    --p-cores 10 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GACTACHVGGGTATCTAATCC \
    --p-error-rate 0.2 \
    --p-discard-untrimmed \
    --o-trimmed-sequences ager_bmr_trimmed_b.qza

# visualize

qiime demux summarize \
  --i-data ager_bmr_trimmed_b.qza \
  --o-visualization ager_bmr_trimmed_b.qzv

In [0]:
# dada2

qiime dada2 denoise-paired \
--i-demultiplexed-seqs ager_bmr_trimmed.qza \
--p-trunc-len-f 266 \
--p-trunc-len-r 201 \
--p-n-threads 10 \
--o-table table_ager_bmr_trimmed.qza \
--o-representative-sequences repseqs_ager_bmr_trimmed.qza \
--o-denoising-stats stats_ager_bmr_trimmed.qza


# visualize

qiime metadata tabulate \
  --m-input-file stats_ager_bmr_trimmed.qza \
  --o-visualization stats_ager_bmr_trimmed.qzv

qiime metadata tabulate \
  --m-input-file table_ager_bmr_trimmed.qza \
  --o-visualization table_ager_bmr_trimmed.qzv

qiime feature-table summarize \
  --i-table table_ager_bmr_trimmed.qza \
  --o-visualization table_summ_ager_bmr_trimmed.qzv \
  --m-sample-metadata-file metadata_ager_bmr.tsv


Metric	Sample \
Number of samples	48 \
Number of features	2,910 \
Total frequency	2,269,961

In [0]:
# dada2_b

qiime dada2 denoise-paired \
--i-demultiplexed-seqs ager_bmr_trimmed_b.qza \
--p-trunc-len-f 268 \
--p-trunc-len-r 233 \
--p-n-threads 7 \
--o-table table_ager_bmr_trimmed_b.qza \
--o-representative-sequences repseqs_ager_bmr_trimmed_b.qza \
--o-denoising-stats stats_ager_bmr_trimmed_b.qza


# visualize

qiime metadata tabulate \
  --m-input-file stats_ager_bmr_trimmed_b.qza \
  --o-visualization stats_ager_bmr_trimmed_b.qzv

qiime metadata tabulate \
  --m-input-file table_ager_bmr_trimmed_b.qza \
  --o-visualization table_ager_bmr_trimmed_b.qzv

qiime feature-table summarize \
  --i-table table_ager_bmr_trimmed_b.qza \
  --o-visualization table_summ_ager_bmr_trimmed_b.qzv \
  --m-sample-metadata-file metadata_ager_bmr.tsv

qiime feature-table tabulate-seqs \
  --i-data repseqs_ager_bmr_trimmed_b.qza  \
  --o-visualization repseqs_ager_bmr_trimmed_b.qzv

**Con vecchio filtro** (vd. dada2 precedente): \
Metric	Sample \
Number of samples	48 \
Number of features	3,303 \
Total frequency	2,750,058



**Con nuovo filtro:** \
Metric	Sample \
Number of samples	48 \
Number of features	2,933 \
Total frequency	2,586,923

Sequence Count \
2933	
Min Length	Max Length	Mean Length	Range	Standard Deviation \
268	486	415.92	218	18.99
\
\
Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	393	402	404	422	427	427	428

In [0]:
# eliminate sequences with length < 400

qiime tools export --input-path repseqs_ager_bmr_trimmed_b.qza --output-path repseqs_ager_bmr_trimmed_b


cat dna-sequences.fasta | paste - - | sed 's/>//' | awk '{ print $1,length($2) }' | sort -k2,2n > stats_seqs.tsv


awk '{ if($2 < 400) { print }}' stats_seqs.tsv > stats_seqs_to_remove.tsv

sed -i -r 's/(\s+)?\S+//2' stats_seqs_to_remove.tsv


# add feature-id

qiime feature-table filter-features \
--i-table table_ager_bmr_trimmed_b.qza \
--m-metadata-file repseqs_ager_bmr_trimmed_b/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-table table_ager_bmr_trimmed_b_filtered.qza


qiime feature-table filter-seqs \
--i-data repseqs_ager_bmr_trimmed_b.qza \
--m-metadata-file repseqs_ager_bmr_trimmed_b/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-data repseqs_ager_bmr_trimmed_b_filtered.qza

# visualization

qiime feature-table tabulate-seqs \
  --i-data repseqs_ager_bmr_trimmed_b_filtered.qza \
  --o-visualization repseqs_ager_bmr_trimmed_b_filtered.qzv

Sequence Count \
2871 \
	

Min Length	Max Length	Mean Length	Range	Standard Deviation
2871	400	486	418.01	86	11.13

Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	401	402	405	422	427	427	428

### Tables

In [0]:
# visualize

qiime feature-table summarize \
  --i-table table_ager_bmr_trimmed_b_filtered.qza \
  --o-visualization table_ager_bmr_trimmed_b_filtered.qzv \
  --m-sample-metadata-file metadata_ager_bmr.tsv

Number of samples	48 \
Number of features	2,871 \
Total frequency	2,585,340

In [0]:
# create BMR feature-table

qiime tools export --input-path table_ager_bmr_trimmed_b_filtered.qza --output-path table_ager_bmr_trimmed_b_filtered


# convert biom

biom convert -i feature-table.biom -o feature-table_ager_bmr_trimmed_b_filtered.tsv --to-tsv

### Taxonomy

In [0]:
# assign taxonomy silva 138 NR99 only bacteria

qiime feature-classifier classify-consensus-vsearch \
  --i-query repseqs_ager_bmr_trimmed_b.qza \
  --i-reference-reads /home/zpl/storage/SILVA_138/Bacteria_SILVA_138_SSURef_NR99.qza \
  --i-reference-taxonomy /home/zpl/storage/SILVA_138/Bacteria_taxonomy_SILVA_138_SSURef_NR99.qza \
  --p-threads 7 \
  --o-classification taxonomy_vsearch_ager_bmr_trimmed_b.qza


# taxabarplot

qiime taxa barplot \
  --i-table table_ager_bmr_trimmed_b.qza \
  --i-taxonomy taxonomy_vsearch_ager_bmr_trimmed_b.qza \
  --m-metadata-file metadata_ager_bmr.tsv \
  --o-visualization taxabarplot_vsearch_ager_bmr_trimmed_b.qzv

In [0]:
# create AGER taxa-table at genus level

qiime taxa collapse \
  --i-table table_ager_bmr_trimmed_b.qza \
  --i-taxonomy taxonomy_vsearch_ager_bmr_trimmed_b.qza \
  --p-level 6 \
  --o-collapsed-table taxa-table_genus_ager_bmr_trimmed_b.qza


# extract

qiime tools export --input-path taxa-table_genus_ager_bmr_trimmed_b.qza --output-path taxa-table_genus_ager_bmr_trimmed_b


# convert biom

biom convert -i feature-table.biom -o taxa-table_genus_ager_bmr_trimmed_b.tsv --to-tsv


# 749 genus 

### Transfer files

In [0]:
cp repseqs_ager_bmr_trimmed_b_filtered.qza /home/zpl/storage/aquaculture_big
cp table_ager_bmr_trimmed_b_filtered.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_ager_bmr_trimmed_b.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_ager_bmr_trimmed_b.tsv /home/zpl/storage/aquaculture_big
cp feature-table_ager_bmr_trimmed_b_filtered.tsv /home/zpl/storage/aquaculture_big

## Hermetia (ANTO)

**Hermetia + Nanoplastics** \
After bbduk cleaning here we have demux statistics: \
\
Demultiplexed sequence counts summary \
Minimum:	8 \
Median:	88585.5 \
Mean:	111386.25609756098 \
Maximum:	360996 \
Total:	9133673

In [0]:
# parto dal demux ed eseguo cutadapt
# pwd: ~/storage/AntoniaB_1128_16S/analysis_for_aq_big

# remove primers

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences demux_pe_aq_pl.qza \
    --p-cores 7 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GACTACHVGGGTATCTAATCC \
    --verbose \
    --p-discard-untrimmed \
    --o-trimmed-sequences analysis_for_aq_big/aq_pl_trimmed.qza

# visualize

qiime demux summarize \
  --i-data analysis_for_aq_big/aq_pl_trimmed.qza \
  --o-visualization analysis_for_aq_big/aq_pl_trimmed.qzv


Minimum:	2 \
Median:	85670.0 \
Mean:	107395.62195121951 \
Maximum:	349816 \
Total:	8806441

In [0]:
# remove primers (b version)

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences demux_pe_aq_pl.qza \
    --p-cores 7 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GACTACHVGGGTATCTAATCC \
    --verbose \
    --p-error-rate 0.2 \
    --p-discard-untrimmed \
    --o-trimmed-sequences analysis_for_aq_big/aq_pl_trimmed_b.qza

# visualize

qiime demux summarize \
  --i-data analysis_for_aq_big/aq_pl_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/aq_pl_trimmed_b.qzv

Minimum:	2 \
Median:	88142.0 \
Mean:	110771.06097560975 \
Maximum:	359206 \
Total:	9083227

In [0]:
# dada2

time qiime dada2 denoise-paired \
--i-demultiplexed-seqs analysis_for_aq_big/aq_pl_trimmed_b.qza  \
--p-trunc-len-f 283 \
--p-trunc-len-r 201 \
--p-n-threads 7 \
--o-representative-sequences analysis_for_aq_big/rep-seqs_aqpl_trimmed_b.qza \
--o-denoising-stats analysis_for_aq_big/stats-dada2_aqpl_trimmed_b.qza \
--o-table analysis_for_aq_big/table_aqpl_trimmed_b.qza \
--verbose


# visualize

qiime metadata tabulate \
  --m-input-file  analysis_for_aq_big/stats-dada2_aqpl_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/stats-dada2_aqpl_trimmed_b.qzv

qiime feature-table tabulate-seqs \
  --i-data analysis_for_aq_big/rep-seqs_aqpl_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/rep-seqs_aqpl_trimmed_b.qzv

Sequence Count \
11630 \


Min Length	Max Length	Mean Length	Range	Standard Deviation \
283	471	417.35	188	13.3

In [0]:
# dada2_b

time qiime dada2 denoise-paired \
--i-demultiplexed-seqs analysis_for_aq_big/aq_pl_trimmed_b.qza  \
--p-trunc-len-f 283 \
--p-trunc-len-r 218 \
--p-n-threads 7 \
--o-representative-sequences analysis_for_aq_big/rep-seqs_b_aqpl_trimmed_b.qza \
--o-denoising-stats analysis_for_aq_big/stats-dada2_b_aqpl_trimmed_b.qza \
--o-table analysis_for_aq_big/table_b_aqpl_trimmed_b.qza \
--verbose


# visualize

qiime metadata tabulate \
  --m-input-file analysis_for_aq_big/stats-dada2_b_aqpl_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/stats-dada2_b_aqpl_trimmed_b.qzv

qiime feature-table tabulate-seqs \
  --i-data analysis_for_aq_big/rep-seqs_b_aqpl_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/rep-seqs_b_aqpl_trimmed_b.qzv

Sequence Count	\
11285 \
\
Min Length	Max Length	Mean Length	Range	Standard Deviation \
283	489	417.41	206	13.37

In [0]:
# remove sequences with lenght < 400

qiime tools export --input-path rep-seqs_b_aqpl_trimmed_b.qza --output-path rep-seqs_b_aqpl_trimmed_b


cat dna-sequences.fasta | paste - - | sed 's/>//' | awk '{ print $1,length($2) }' | sort -k2,2n > stats_seqs.tsv


awk '{ if($2 < 400) { print }}' stats_seqs.tsv > stats_seqs_to_remove.tsv

sed -i -r 's/(\s+)?\S+//2' stats_seqs_to_remove.tsv


# add feature-id

qiime feature-table filter-features \
--i-table table_b_aqpl_trimmed_b.qza \
--m-metadata-file rep-seqs_b_aqpl_trimmed_b/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-table table_b_aqpl_trimmed_b_filtered.qza


qiime feature-table filter-seqs \
--i-data rep-seqs_b_aqpl_trimmed_b.qza \
--m-metadata-file rep-seqs_b_aqpl_trimmed_b/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-data rep-seqs_b_aqpl_trimmed_b_filtered.qza

# visualization

qiime feature-table tabulate-seqs \
  --i-data rep-seqs_b_aqpl_trimmed_b_filtered.qza \
  --o-visualization rep-seqs_b_aqpl_trimmed_b_filtered.qzv

Sequence Count \
11129 \

Min Length	Max Length	Mean Length	Range	Standard Deviation
400	489	418.13	89	11.03

Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	402	402	404	422	427	428	429

### Tables

In [0]:
# keep only AGER data

qiime feature-table filter-samples \
  --i-table analysis_for_aq_big/table_b_aqpl_trimmed_b_filtered.qza \
  --m-metadata-file mapping_file_CTGB-2019-01175_1128.tsv \
  --p-where "Project='AGER'" \
  --o-filtered-table analysis_for_aq_big/table_b_aqpl_trimmed_b_filtered_AGER.qza


# visualize

qiime feature-table summarize \
  --i-table analysis_for_aq_big/table_b_aqpl_trimmed_b_filtered_AGER.qza \
  --o-visualization analysis_for_aq_big/table_b_aqpl_trimmed_b_filtered_AGER.qzv \
  --m-sample-metadata-file mapping_file_CTGB-2019-01175_1128.tsv


Number of samples	51 \
Number of features	10,896 \
Total frequency	5,012,538

In [0]:
# create AGER rep-seq file

qiime feature-table filter-seqs \
  --i-data analysis_for_aq_big/rep-seqs_b_aqpl_trimmed_b_filtered.qza \
  --i-table analysis_for_aq_big/table_b_aqpl_trimmed_b_filtered_AGER.qza \
  --o-filtered-data analysis_for_aq_big/rep-seqs_b_aqpl_trimmed_b_filtered_AGER.qza

In [0]:
# create AGER feature-table

qiime tools export --input-path table_b_aqpl_trimmed_b_filtered_AGER.qza --output-path table_b_aqpl_trimmed_b_filtered_AGER

# convert biom

biom convert -i feature-table.biom -o feature-table_b_aqpl_trimmed_b_filtered_AGER.tsv --to-tsv

### Taxonomy

In [0]:
# assign taxonomy silva 138 NR99 only bacteria

qiime feature-classifier classify-consensus-vsearch \
  --i-query rep-seqs_b_aqpl_trimmed_b_filtered.qza \
  --i-reference-reads /home/zpl/storage/SILVA_138/Bacteria_SILVA_138_SSURef_NR99.qza \
  --i-reference-taxonomy /home/zpl/storage/SILVA_138/Bacteria_taxonomy_SILVA_138_SSURef_NR99.qza \
  --p-threads 5 \
  --o-classification taxonomy_vsearch_b_aqpl_trimmed_b_filtered.qza


# taxabarplot

qiime taxa barplot \
  --i-table analysis_for_aq_big/table_b_aqpl_trimmed_b_filtered_AGER.qza \
  --i-taxonomy analysis_for_aq_big/taxonomy_vsearch_b_aqpl_trimmed_b_filtered.qza \
  --m-metadata-file mapping_file_CTGB-2019-01175_1128.tsv \
  --o-visualization analysis_for_aq_big/taxabarplot_vsearch_b_aqpl_trimmed_b_filtered_AGER.qzv

In [0]:
# create AGER taxa-table at genus level

qiime taxa collapse \
  --i-table table_b_aqpl_trimmed_b_filtered_AGER.qza \
  --i-taxonomy taxonomy_vsearch_b_aqpl_trimmed_b_filtered.qza \
  --p-level 6 \
  --o-collapsed-table taxa-table_genus_b_aqpl_trimmed_b_filtered_AGER.qza

In [0]:
# extract taxa table

qiime tools export --input-path taxa-table_genus_b_aqpl_trimmed_b_filtered_AGER.qza --output-path taxa-table_genus_b_aqpl_trimmed_b_filtered_AGER

# convert biom

biom convert -i feature-table.biom -o taxa-table_genus_b_aqpl_trimmed_b_filtered_AGER.tsv --to-tsv

### Transfer files

In [0]:
cp rep-seqs_b_aqpl_trimmed_b_filtered_AGER.qza /home/zpl/storage/aquaculture_big
cp table_b_aqpl_trimmed_b_filtered_AGER.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_b_aqpl_trimmed_b_filtered_AGER.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_b_aqpl_trimmed_b_filtered_AGER.tsv /home/zpl/storage/aquaculture_big
cp feature-table_b_aqpl_trimmed_b_filtered_AGER.tsv /home/zpl/storage/aquaculture_big

### Diversity analysis

In [0]:
# diversity analysis della table ager_bmr 

qiime diversity core-metrics \
  --i-table table_aqtz_trimmed_b_filtered_AGER.qza \
  --p-sampling-depth 14000 \
  --m-metadata-file /home/groot/storageBioinfoLAB/aquaculture_big/metadata_aquaculture_big_v3v4.tsv \
  --output-dir div_aqtz_trimmed_b_filtered_AGER

## Tenebrio (ANTO)

Minimum:	3376 \
Median:	173278.5 \
Mean:	234771.28125 \
Maximum:	2895415 \
Total:	15025362

In [0]:
# parto dal demux ed eseguo cutadapt
# pwd: /home/zpl/storage/anto_aquaculture/files/analysis_for_aq_big

# remove primers

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences demux-pe_aqtz.qza \
    --p-cores 1 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GACTACHVGGGTATCTAATCC \
    --verbose \
    --p-discard-untrimmed \
    --o-trimmed-sequences analysis_for_aq_big/aq_tz_trimmed.qza

# visualize

qiime demux summarize \
  --i-data analysis_for_aq_big/aq_tz_trimmed.qza \
  --o-visualization analysis_for_aq_big/aq_tz_trimmed.qzv


Minimum:	3260 \
Median:	153779.5 \
Mean:	229950.125 \
Maximum:	2872078 \
Total:	14716808

In [0]:
# remove primers

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences demux-pe_aqtz.qza \
    --p-cores 1 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GACTACHVGGGTATCTAATCC \
    --verbose \
    --p-error-rate 0.2 \
    --p-discard-untrimmed \
    --o-trimmed-sequences analysis_for_aq_big/aq_tz_trimmed_b.qza

# visualize

qiime demux summarize \
  --i-data analysis_for_aq_big/aq_tz_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/aq_tz_trimmed_b.qzv


Minimum:	3297 \
Median:	166929.0 \
Mean:	233054.703125 \
Maximum:	2887311 \
Total:	14915501

In [0]:
# export TMPDIR=/home/zooplantlab/tmp

# dada2

time qiime dada2 denoise-paired \
--i-demultiplexed-seqs analysis_for_aq_big/aq_tz_trimmed_b.qza  \
--p-trunc-len-f 280 \
--p-trunc-len-r 220 \
--p-n-threads 8 \
--o-representative-sequences analysis_for_aq_big/rep-seqs_aqtz_trimmed_b.qza \
--o-denoising-stats analysis_for_aq_big/stats-dada2_aqtz_trimmed_b.qza \
--o-table analysis_for_aq_big/table_aqtz_trimmed_b.qza \
--verbose


# visualize

qiime metadata tabulate \
  --m-input-file analysis_for_aq_big/stats-dada2_aqtz_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/stats-dada2_aqtz_trimmed_b.qzv

qiime feature-table tabulate-seqs \
  --i-data analysis_for_aq_big/rep-seqs_aqtz_trimmed_b.qza \
  --o-visualization analysis_for_aq_big/rep-seqs_aqtz_trimmed_b.qzv

Sequence Count \
34395	\
Min Length	Max Length	Mean Length	Range	Standard Deviation \
280	488	411.81	208	24.75

Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	306	402	403	421	427	428	429

In [0]:
# remove sequences with lenght < 400

qiime tools export --input-path rep-seqs_aqtz_trimmed_b.qza --output-path rep-seqs_aqtz_trimmed_b


cat dna-sequences.fasta | paste - - | sed 's/>//' | awk '{ print $1,length($2) }' | sort -k2,2n > stats_seqs.tsv


awk '{ if($2 < 400) { print }}' stats_seqs.tsv > stats_seqs_to_remove.tsv

sed -i -r 's/(\s+)?\S+//2' stats_seqs_to_remove.tsv


# add feature-id con nano

qiime feature-table filter-features \
--i-table table_aqtz_trimmed_b.qza  \
--m-metadata-file rep-seqs_aqtz_trimmed_b/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-table table_aqtz_trimmed_b_filtered.qza


qiime feature-table filter-seqs \
--i-data rep-seqs_aqtz_trimmed_b.qza \
--m-metadata-file rep-seqs_aqtz_trimmed_b/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-data rep-seqs_aqtz_trimmed_b_filtered.qza

# visualization

qiime feature-table tabulate-seqs \
  --i-data rep-seqs_aqtz_trimmed_b_filtered.qza \
  --o-visualization rep-seqs_aqtz_trimmed_b_filtered.qzv

Sequence Count \
32206 \	
Min Length	Max Length	Mean Length	Range	Standard Deviation \
400	488	416.78	88	11.74 

\
Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	402	402	404	422	427	428	429

### Tables

In [0]:
# keep only AGER data

qiime feature-table filter-samples \
  --i-table analysis_for_aq_big/table_aqtz_trimmed_b_filtered.qza \
  --m-metadata-file metadata_aqtz.tsv  \
  --p-where "Project='AGER'" \
  --o-filtered-table analysis_for_aq_big/table_aqtz_trimmed_b_filtered_AGER.qza


# visualize

qiime feature-table summarize \
  --i-table analysis_for_aq_big/table_aqtz_trimmed_b_filtered_AGER.qza \
  --o-visualization analysis_for_aq_big/table_aqtz_trimmed_b_filtered_AGER.qzv \
  --m-sample-metadata-file metadata_aqtz.tsv

Number of samples	57 \
Number of features	13,528 \
Total frequency	6,841,821

In [0]:
# create AGER rep-seq file

qiime feature-table filter-seqs \
  --i-data analysis_for_aq_big/rep-seqs_aqtz_trimmed_b_filtered.qza \
  --i-table analysis_for_aq_big/table_aqtz_trimmed_b_filtered_AGER.qza \
  --o-filtered-data analysis_for_aq_big/rep-seqs_aqtz_trimmed_b_filtered_AGER.qza

In [0]:
# create AGER feature-table

qiime tools export --input-path table_aqtz_trimmed_b_filtered_AGER.qza --output-path table_aqtz_trimmed_b_filtered_AGER

# convert biom

biom convert -i feature-table.biom -o feature-table_aqtz_trimmed_b_filtered_AGER.tsv --to-tsv

### Taxonomy

In [0]:
# assign taxonomy silva 138 NR99 only bacteria

time qiime feature-classifier classify-consensus-vsearch \
  --i-query rep-seqs_aqtz_trimmed_b_filtered.qza \
  --i-reference-reads /home/zpl/storage/SILVA_138/Bacteria_SILVA_138_SSURef_NR99.qza \
  --i-reference-taxonomy /home/zpl/storage/SILVA_138/Bacteria_taxonomy_SILVA_138_SSURef_NR99.qza \
  --p-threads 10 \
  --o-classification taxonomy_vsearch_aqtz_trimmed_b_filtered.qza


# taxabarplot

qiime taxa barplot \
  --i-table analysis_for_aq_big/table_aqtz_trimmed_b_filtered_AGER.qza  \
  --i-taxonomy analysis_for_aq_big/taxonomy_vsearch_aqtz_trimmed_b_filtered.qza \
  --m-metadata-file metadata_aqtz.tsv \
  --o-visualization analysis_for_aq_big/taxabarplot_vsearch_aqtz_trimmed_b_filtered.qzv

In [0]:
# create AGER taxa-table at genus level

qiime taxa collapse \
  --i-table table_aqtz_trimmed_b_filtered_AGER.qza \
  --i-taxonomy taxonomy_vsearch_aqtz_trimmed_b_filtered.qza \
  --p-level 6 \
  --o-collapsed-table taxa-table_genus_aqtz_trimmed_b_filtered_AGER.qza

In [0]:
# extract taxa table

qiime tools export --input-path taxa-table_genus_aqtz_trimmed_b_filtered_AGER.qza --output-path taxa-table_genus_aqtz_trimmed_b_filtered_AGER

# convert biom

biom convert -i feature-table.biom -o taxa-table_genus_aqtz_trimmed_b_filtered_AGER.tsv --to-tsv

### Transfer files

In [0]:
cp rep-seqs_aqtz_trimmed_b_filtered_AGER.qza /home/zpl/storage/aquaculture_big
cp table_aqtz_trimmed_b_filtered_AGER.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_aqtz_trimmed_b_filtered_AGER.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_aqtz_trimmed_b_filtered_AGER.tsv /home/zpl/storage/aquaculture_big
cp feature-table_aqtz_trimmed_b_filtered_AGER.tsv /home/zpl/storage/aquaculture_big

### Diversity analysis

In [0]:
# diversity analysis della table aqtz

qiime diversity core-metrics \
  --i-table table_aqtz_trimmed_b_filtered_AGER.qza \
  --p-sampling-depth 14000 \
  --m-metadata-file /home/groot/storageBioinfoLAB/aquaculture_big/metadata_aquaculture_big_v3v4.tsv \
  --output-dir div_aqtz_trimmed_b_filtered_AGER

# MYSUSHI

Minimum:	24940 \
Median:	71465.0 \
Mean:	73274.25531914894 \
Maximum:	133543 \
Total:	6887780

In [0]:
# parto dal demux ed eseguo cutadapt
# pwd: /home/zpl/storage/mysushi_bioinfo/analysis_for_aq_big

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences mysushi_152253.qza \
    --p-cores 1 \
    --p-front-f GTGCCAGCMGCCGCGGTAA \
    --p-front-r GGACTACHVGGGTWTCTAAT \
    --verbose \
    --p-discard-untrimmed \
    --o-trimmed-sequences analysis_for_aq_big/mysushi_152253_trimmed.qza


# visualize

qiime demux summarize \
  --i-data analysis_for_aq_big/mysushi_152253_trimmed.qza \
  --o-visualization analysis_for_aq_big/mysushi_152253_trimmed.qzv


Minimum:	24740 \
Median:	70820.0 \
Mean:	72617.14893617021 \
Maximum:	132374 \
Total:	6826012

In [0]:
# dada2

qiime dada2 denoise-paired \
  --i-demultiplexed-seqs analysis_for_aq_big/mysushi_152253_trimmed.qza \
  --p-trunc-len-f 242 \
  --p-trunc-len-r 174 \
  --p-n-threads 7 \
  --o-representative-sequences analysis_for_aq_big/rep-seqs_mysushi_152253_trimmed.qza \
  --o-denoising-stats analysis_for_aq_big/stats-dada2_mysushi_152253_trimmed.qza \
  --o-table analysis_for_aq_big/table_mysushi_152253_trimmed.qza \
  --verbose


# visualize

qiime feature-table summarize  \
  --m-sample-metadata-file map_mysushi.tsv \
  --i-table analysis_for_aq_big/table_mysushi_152253_trimmed.qza \
  --o-visualization analysis_for_aq_big/table_mysushi_152253_trimmed.qzv

qiime metadata tabulate \
  --m-input-file analysis_for_aq_big/stats-dada2_mysushi_152253_trimmed.qza\
  --o-visualization analysis_for_aq_big/stats-dada2_mysushi_152253_trimmed.qzv

qiime feature-table tabulate-seqs \
  --i-data analysis_for_aq_big/rep-seqs_mysushi_152253_trimmed.qza \
  --o-visualization analysis_for_aq_big/rep-seqs_mysushi_152253_trimmed.qzv

# tenuto il 30-40% delle sequenze
# non è molto
# siccome questa è un'analisi tangenziale teniamo così al massimo ricorriamo


Sequence Count \
3283	\
Min Length	Max Length	Mean Length	Range	Standard Deviation \
242	404	267.42	162	35.18

Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	242	250	253	253	254	332	384

## Tables

In [0]:
# visualize

qiime feature-table summarize \
  --i-table analysis_for_aq_big/table_mysushi_152253_trimmed.qza \
  --o-visualization analysis_for_aq_big/table_mysushi_152253_trimmed.qzv \
  --m-sample-metadata-file map_mysushi.tsv 

In [0]:
# create feature-table

qiime tools export --input-path table_mysushi_152253_trimmed.qza --output-path table_mysushi_152253_trimmed

# convert biom

biom convert -i feature-table.biom -o feature-table_mysushi_152253_trimmed.tsv --to-tsv

##Taxonomy

In [0]:
# assign taxonomy silva 138 NR99 only bacteria

time qiime feature-classifier classify-consensus-vsearch \
  --i-query rep-seqs_mysushi_152253_trimmed.qza \
  --i-reference-reads /home/zpl/storage/SILVA_138/Bacteria_SILVA_138_SSURef_NR99.qza \
  --i-reference-taxonomy /home/zpl/storage/SILVA_138/Bacteria_taxonomy_SILVA_138_SSURef_NR99.qza \
  --p-threads 7 \
  --o-classification taxonomy_vsearch_mysushi_152253_trimmed.qza


# da lanciare
# taxabarplot

qiime taxa barplot \
  --i-table analysis_for_aq_big/table_mysushi_152253_trimmed.qza  \
  --i-taxonomy analysis_for_aq_big/taxonomy_vsearch_mysushi_152253_trimmed.qza \
  --m-metadata-file map_mysushi.tsv  \
  --o-visualization analysis_for_aq_big/taxabarplot_vsearch_mysushi_152253_trimmed.qzv

In [0]:
# da lanciare
# create mysushi taxa-table at genus level

qiime taxa collapse \
  --i-table table_mysushi_152253_trimmed.qza \
  --i-taxonomy taxonomy_vsearch_mysushi_152253_trimmed.qza \
  --p-level 6 \
  --o-collapsed-table taxa-table_genus_mysushi_152253_trimmed.qza

In [0]:
# da lanciare
# extract taxa table

qiime tools export --input-path taxa-table_genus_mysushi_152253_trimmed.qza --output-path taxa-table_genus_mysushi_152253_trimmed

# convert biom

biom convert -i feature-table.biom -o taxa-table_genus_genus_mysushi_152253_trimmed.tsv --to-tsv

##Transfer files

In [0]:
cp 
cp

# BAZZOCCHI

Minimum:	4 \
Median:	189017.0 \
Mean:	144476.76923076922 \
Maximum:	404182 \
Total:	3756396

In [0]:
# pwd : /home/zpl/storage/Proj_1008_16S/raw_file_Pr1008/bazzocchi_raw/results_baz_acult
# parto da demultiplex Baz_acult.qza

# remove primers

qiime cutadapt trim-paired  \
    --i-demultiplexed-sequences Baz_acult.qza \
    --p-cores 1 \
    --p-front-f CCTACGGGNGGCWGCAG \
    --p-front-r GACTACHVGGGTATCTAATCC \
    --verbose \
    --p-error-rate 0.2 \
    --p-discard-untrimmed \
    --o-trimmed-sequences analysis_for_aq_big/demux_bazz_acult_trimmed.qza


# visualize

qiime demux summarize \
  --i-data analysis_for_aq_big/demux_bazz_acult_trimmed.qza \
  --o-visualization analysis_for_aq_big/demux_bazz_acult_trimmed.qzv

Minimum:	3 \
Median:	187811.0 \
Mean:	143603.92307692306 \
Maximum:	401650 \
Total:	3733702

In [0]:
# vedo stats pipeline Anna
# pwd: ~/storage/Proj_1008_16S/raw_file_Pr1008/bazzocchi_raw/results_baz_acult

qiime metadata tabulate \
  --m-input-file denoising-stats_baz_acult.qza \
  --o-visualization denoising-stats_baz_acult_anna.qzv

# le sequenze si dimezzano!

In [0]:
# dada2

time qiime dada2 denoise-paired \
--i-demultiplexed-seqs analysis_for_aq_big/demux_bazz_acult_trimmed.qza   \
--p-trunc-len-f 271 \
--p-trunc-len-r 215 \
--p-n-threads 16 \
--o-representative-sequences analysis_for_aq_big/rep-seqs_bazz_acult_trimmed.qza \
--o-denoising-stats analysis_for_aq_big/stats-dada2_bazz_acult_trimmed.qza \
--o-table analysis_for_aq_big/table_bazz_acult_trimmed.qza \
--verbose


#1) Filtering The filter removed all reads: /tmp/tmpj54vj461/filt_f/TA_C_UP_3_RplusI_045minus022_S47_L001_R1_001.fastq.gz and /tmp/tmpj54vj461/filt_r/TA_C_UP_3_RplusI_045minus022_S47_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /tmp/tmpj54vj461/filt_f/TA_C_UP_3_RplusI_3_S46_L001_R1_001.fastq.gz and /tmp/tmpj54vj461/filt_r/TA_C_UP_3_RplusI_3_S46_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /tmp/tmpj54vj461/filt_f/Bianco_3EXT_S41_L001_R1_001.fastq.gz and /tmp/tmpj54vj461/filt_r/Bianco_3EXT_S41_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /tmp/tmpj54vj461/filt_f/TA_OUT_2_045minus022_S45_L001_R1_001.fastq.gz and /tmp/tmpj54vj461/filt_r/TA_OUT_2_045minus022_S45_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /tmp/tmpj54vj461/filt_f/WE_1_3_S42_L001_R1_001.fastq.gz and /tmp/tmpj54vj461/filt_r/WE_1_3_S42_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /tmp/tmpj54vj461/filt_f/WE_1_045minus022_S43_L001_R1_001.fastq.gz and /tmp/tmpj54vj461/filt_r/WE_1_045minus022_S43_L001_R2_001.fastq.gz not written.


# visualize

qiime feature-table summarize  \
  --m-sample-metadata-file baz_acult_map.tsv \
  --i-table analysis_for_aq_big/table_bazz_acult_trimmed.qza \
  --o-visualization analysis_for_aq_big/table_bazz_acult_trimmed.qzv

qiime metadata tabulate \
  --m-input-file analysis_for_aq_big/stats-dada2_bazz_acult_trimmed.qza \
  --o-visualization analysis_for_aq_big/stats-dada2_bazz_acult_trimmed.qzv

qiime feature-table tabulate-seqs \
  --i-data analysis_for_aq_big/rep-seqs_bazz_acult_trimmed.qza \
  --o-visualization analysis_for_aq_big/rep-seqs_bazz_acult_trimmed.qzv

# rese bassissime! 30-40%


Sequence Count 3987	\
Min Length	Max Length	Mean Length	Range	Standard Deviation \
271	461	415.57	190	17.63

Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	401	402	404	422	427	427	428

In [0]:
# dada2 b

# export TMPDIR=/home/zooplantlab/tmp

time qiime dada2 denoise-paired \
--i-demultiplexed-seqs analysis_for_aq_big/demux_bazz_acult_trimmed.qza   \
--p-trunc-len-f 271 \
--p-trunc-len-r 215 \
--p-n-threads 16 \
--p-max-ee-f 2.5 \
--p-max-ee-r 2.5 \
--o-representative-sequences analysis_for_aq_big/rep-seqs_b_bazz_acult_trimmed.qza \
--o-denoising-stats analysis_for_aq_big/stats-dada2_b_bazz_acult_trimmed.qza \
--o-table analysis_for_aq_big/table_b_bazz_acult_trimmed.qza \
--verbose


#1) Filtering The filter removed all reads: /home/zooplantlab/tmp/tmpb_si4qqj/filt_f/Bianco_3EXT_S41_L001_R1_001.fastq.gz and /home/zooplantlab/tmp/tmpb_si4qqj/filt_r/Bianco_3EXT_S41_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /home/zooplantlab/tmp/tmpb_si4qqj/filt_f/TA_C_UP_3_RplusI_3_S46_L001_R1_001.fastq.gz and /home/zooplantlab/tmp/tmpb_si4qqj/filt_r/TA_C_UP_3_RplusI_3_S46_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /home/zooplantlab/tmp/tmpb_si4qqj/filt_f/TA_C_UP_3_RplusI_045minus022_S47_L001_R1_001.fastq.gz and /home/zooplantlab/tmp/tmpb_si4qqj/filt_r/TA_C_UP_3_RplusI_045minus022_S47_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /home/zooplantlab/tmp/tmpb_si4qqj/filt_f/TA_OUT_2_045minus022_S45_L001_R1_001.fastq.gz and /home/zooplantlab/tmp/tmpb_si4qqj/filt_r/TA_OUT_2_045minus022_S45_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /home/zooplantlab/tmp/tmpb_si4qqj/filt_f/WE_1_3_S42_L001_R1_001.fastq.gz and /home/zooplantlab/tmp/tmpb_si4qqj/filt_r/WE_1_3_S42_L001_R2_001.fastq.gz not written.
#The filter removed all reads: /home/zooplantlab/tmp/tmpb_si4qqj/filt_f/WE_1_045minus022_S43_L001_R1_001.fastq.gz and /home/zooplantlab/tmp/tmpb_si4qqj/filt_r/WE_1_045minus022_S43_L001_R2_001.fastq.gz not written.

# visualize

qiime metadata tabulate \
  --m-input-file analysis_for_aq_big/stats-dada2_b_bazz_acult_trimmed.qza \
  --o-visualization analysis_for_aq_big/stats-dada2_b_bazz_acult_trimmed.qzv

qiime feature-table tabulate-seqs \
  --i-data analysis_for_aq_big/rep-seqs_b_bazz_acult_trimmed.qza \
  --o-visualization analysis_for_aq_big/rep-seqs_b_bazz_acult_trimmed.qzv

# non è cambiato molto
# tengo il file di prima

In [0]:
# remove sequences with lenght < 400

qiime tools export --input-path rep-seqs_bazz_acult_trimmed.qza --output-path rep-seqs_bazz_acult_trimmed


cat dna-sequences.fasta | paste - - | sed 's/>//' | awk '{ print $1,length($2) }' | sort -k2,2n > stats_seqs.tsv


awk '{ if($2 < 400) { print }}' stats_seqs.tsv > stats_seqs_to_remove.tsv

sed -i -r 's/(\s+)?\S+//2' stats_seqs_to_remove.tsv


# add feature-id con nano

qiime feature-table filter-features \
--i-table table_bazz_acult_trimmed.qza  \
--m-metadata-file rep-seqs_bazz_acult_trimmed/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-table table_bazz_acult_trimmed_filtered.qza

qiime feature-table filter-seqs \
--i-data rep-seqs_bazz_acult_trimmed.qza \
--m-metadata-file rep-seqs_bazz_acult_trimmed/stats_seqs_to_remove.tsv \
--p-exclude-ids \
--o-filtered-data rep-seqs_bazz_acult_trimmed_filtered.qza


# visualization

qiime feature-table tabulate-seqs \
  --i-data rep-seqs_bazz_acult_trimmed_filtered.qza \
  --o-visualization rep-seqs_bazz_acult_trimmed_filtered.qzv


Sequence Count \
3926 \
Min Length	Max Length	Mean Length	Range	Standard Deviation \
401	461	417.16	60	10.58

\
Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	402	402	404	422	427	427	428

### Tables

In [0]:
# visualize

qiime feature-table summarize \
  --i-table analysis_for_aq_big/table_bazz_acult_trimmed_filtered.qza \
  --o-visualization analysis_for_aq_big/table_bazz_acult_trimmed_filtered.qzv \
  --m-sample-metadata-file baz_acult_map.tsv

In [0]:
# create feature-table

qiime tools export --input-path table_bazz_acult_trimmed_filtered.qza --output-path table_bazz_acult_trimmed_filtered

# convert biom

biom convert -i feature-table.biom -o feature-table_bazz_acult_trimmed_filtered.tsv --to-tsv

### Taxonomy

In [0]:
# assign taxonomy silva 138 NR99 only bacteria

time qiime feature-classifier classify-consensus-vsearch \
  --i-query rep-seqs_bazz_acult_trimmed_filtered.qza  \
  --i-reference-reads /home/zpl/storage/SILVA_138/Bacteria_SILVA_138_SSURef_NR99.qza \
  --i-reference-taxonomy /home/zpl/storage/SILVA_138/Bacteria_taxonomy_SILVA_138_SSURef_NR99.qza \
  --p-threads 7 \
  --o-classification taxonomy_vsearch_bazz_acult_trimmed_filtered.qza


# taxabarplot

qiime taxa barplot \
  --i-table analysis_for_aq_big/table_bazz_acult_trimmed_filtered.qza \
  --i-taxonomy analysis_for_aq_big/taxonomy_vsearch_bazz_acult_trimmed_filtered.qza \
  --m-metadata-file baz_acult_map.tsv \
  --o-visualization analysis_for_aq_big/taxabarplot_vsearch_bazz_acult_trimmed_filtered.qza

In [0]:
# create bazzocchi taxa-table at genus level

qiime taxa collapse \
  --i-table table_bazz_acult_trimmed_filtered.qza \
  --i-taxonomy taxonomy_vsearch_bazz_acult_trimmed_filtered.qza \
  --p-level 6 \
  --o-collapsed-table taxa-table_genus_bazz_acult_trimmed_filtered.qza

In [0]:
# extract taxa table

qiime tools export --input-path taxa-table_genus_bazz_acult_trimmed_filtered.qza --output-path taxa-table_genus_bazz_acult_trimmed_filtered

# convert biom

biom convert -i feature-table.biom -o taxa-table_genus_bazz_acult_trimmed_filtered.tsv --to-tsv

### Transfer files

In [0]:
cp rep-seqs_bazz_acult_trimmed_filtered.qza /home/zpl/storage/aquaculture_big
cp table_bazz_acult_trimmed_filtered.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_bazz_acult_trimmed_filtered.qza /home/zpl/storage/aquaculture_big
cp taxa-table_genus_bazz_acult_trimmed_filtered.tsv /home/zpl/storage/aquaculture_big
cp feature-table_bazz_acult_trimmed_filtered.tsv /home/zpl/storage/aquaculture_big

# Merge Aquaculture Projects

In [0]:
# merge feature tables and rep seqs (without taxonomy assignment)
# add metadata_aquaculture_big.tsv

# pwd: /home/zpl/storage/aquaculture_big

# AGER_BMR
# ANTO (Hermetia)
# ANTO (Tenebrio)
# BAZZOCCHI
# MYSUSHI (merge solo della taxa-table per diversità marcatore)

In [0]:
# merge tables

qiime feature-table merge \
 --i-tables table_ager_bmr_trimmed_b_filtered.qza  \
 --i-tables table_aqtz_trimmed_b_filtered_AGER.qza \
 --i-tables table_b_aqpl_trimmed_b_filtered_AGER.qza \
 --i-tables table_bazz_acult_trimmed_filtered.qza \
 --o-merged-table merged_tables/table_aquaculture_big_v3v4.qza

# visualize

qiime feature-table summarize \
  --i-table table_aquaculture_big_v3v4.qza \
  --o-visualization table_aquaculture_big_v3v4.qzv \
  --m-sample-metadata-file ~/storageBioinfoLAB/aquaculture_big/metadata_aquaculture_big_v3v4.tsv

Number of samples	172 \
Number of features	24,143 \
Total frequency	15,757,796 

In [0]:
# merge rep-seqs 

qiime feature-table merge-seqs \
--i-data repseqs_ager_bmr_trimmed_b_filtered.qza \
--i-data rep-seqs_aqtz_trimmed_b_filtered_AGER.qza \
--i-data rep-seqs_b_aqpl_trimmed_b_filtered_AGER.qza \
--i-data rep-seqs_bazz_acult_trimmed_filtered.qza \
--o-merged-data merged_tables/rep-seqs_aquaculture_big_v3v4.qza

# visualize
qiime feature-table tabulate-seqs \
--i-data rep-seqs_aquaculture_big_v3v4.qza \
--o-visualization rep-seqs_aquaculture_big_v3v4.qzv


Sequence Count \
24143 \
	Min Length	Max Length	Mean Length	Range	Standard Deviation \
400	489	418.56	89	11.66


Percentile:	2%	9%	25%	50%	75%	91%	98% \
Length* (nts):	402	402	405	422	427	428	429

In [0]:
# taxonomy tables

qiime feature-table merge \
--i-tables taxa-table_genus_ager_bmr_trimmed_b.qza \
--i-tables taxa-table_genus_aqtz_trimmed_b_filtered_AGER.qza \
--i-tables taxa-table_genus_b_aqpl_trimmed_b_filtered_AGER.qza \
--i-tables taxa-table_genus_bazz_acult_trimmed_filtered.qza \
--o-merged-table merged_tables/taxa-table_aquaculture_big_v3v4.qza

# visualize

qiime feature-table summarize \
  --i-table taxa-table_aquaculture_big_v3v4.qza \
  --o-visualization taxa-table_aquaculture_big_v3v4.qzv \
  --m-sample-metadata-file ~/storageBioinfoLAB/aquaculture_big/metadata_aquaculture_big_v3v4.tsv

In [0]:
# taxonomy_vsearch_bazz_acult_trimmed_filtered.qza
# taxonomy_vsearch_aqtz_trimmed_b_filtered.qza
# taxonomy_vsearch_ager_bmr_trimmed_b.qza
# taxonomy_vsearch_b_aqpl_trimmed_b_filtered.qza

# merge taxonomy

qiime feature-table merge-taxa \
--i-data ~/storageBioinfoLAB/Proj_1008_16S/raw_file_Pr1008/bazzocchi_raw/results_baz_acult/analysis_for_aq_big/taxonomy_vsearch_bazz_acult_trimmed_filtered.qza \
--i-data ~/storageBioinfoLAB/anto_aquaculture/files/analysis_for_aq_big/taxonomy_vsearch_aqtz_trimmed_b_filtered.qza \
--i-data ~/storageBioinfoLAB/anto_aquaculture/Ager_bmr_analysis/taxonomy_vsearch_ager_bmr_trimmed_b.qza \
--i-data ~/storageBioinfoLAB/AntoniaB_1128_16S/analysis_for_aq_big/taxonomy_vsearch_b_aqpl_trimmed_b_filtered.qza \
--o-merged-data taxonomy_aquaculture_big_v3v4.qza


# visualize
qiime metadata tabulate \
  –-m-input-file taxonomy_aquaculture_big_v3v4.qza \
  --o-visualization taxonomy_aquaculture_big_v3v4.qzv 


# taxa bar plot

qiime taxa barplot \
  --i-table table_aquaculture_big_v3v4.qza \
  --i-taxonomy taxonomy_aquaculture_big_v3v4.qza \
  --m-metadata-file ~/storageBioinfoLAB/aquaculture_big/metadata_aquaculture_big_v3v4.tsv \
  --o-visualization taxabarplot_aquaculture_big_v3v4.qzv


## Diversity analysis

In [0]:
# diversity
#### scaricare le pcoa per Anto



qiime diversity core-metrics \
  --i-table table_aquaculture_big_v3v4.qza \
  --p-sampling-depth 14000\
  --m-metadata-file /home/groot/storageBioinfoLAB/aquaculture_big/metadata_aquaculture_big_v3v4.tsv \
  --output-dir div_14k_table_aquaculture_big_v3v4

# scp

scp groot@10.99.226.162:~/storageBioinfoLAB/aquaculture_big/merged_tables/div_14k_table_aquaculture_big_v3v4/jaccard_emperor.qzv ~/Documents/PHD/aquaculture_big/div_14k_table_aquaculture_big_v3v4

scp groot@10.99.226.162:~/storageBioinfoLAB/aquaculture_big/merged_tables/div_14k_table_aquaculture_big_v3v4/bray_curtis_emperor.qzv ~/Documents/PHD/aquaculture_big/div_14k_table_aquaculture_big_v3v4

In [0]:
# diversity per genus
#### scaricare le pcoa per Anto

qiime diversity core-metrics \
  --i-table taxa-table_aquaculture_big_v3v4.qza \
  --p-sampling-depth 14000\
  --m-metadata-file /home/groot/storageBioinfoLAB/aquaculture_big/metadata_aquaculture_big_v3v4.tsv \
  --output-dir div_14k_taxa_table_aquaculture_big_v3v4

##check midichloria sequence

16SmidiFish_RTF	5'- GCGGTTATCTGGGCAGTC -3'

16SmidiFish_RTR    5'- TGCGACACCGAAACCTAAG -3'
 
L'accession number dell'unica sequenza di MLO che avevamo disponibile in precedenza è quella pubblicata da Lloyd et al. (2008): EU555284. Blastando la sequenza ottenuta dai colleghi di Pisa, si ottiene un'identità del 98% rispetto alla sequenza della MLO del paper della Lloyd



> mlo SEQUENZA ASSEMBLATA
AGGTGAAAGATTTATCGTTCCAAGACGAGCCCATGCAAGATTAGTTAGTTGGTAAGGTAATTGCTTACCAAGGCAACGATCTTTAGTTGGTCTGAGAGGATGATCAGCCACACTGGAACTGAGACACGTTCCAGATTCTTACGGGAGGCACCAGTGGGGAATATTGGACAATGGGCGCAAGCCTGATCCAGCGACGCCGCGTGAGTGATGAAGCCCTTAGGGTTGTAAAGCTCTTTTAGTGGGGAAGATAATGACGGTACCCACAGAAAAAGCCCCGGCTAACTTCGTGCCANCAGCCGCGGTAATACGAAGGGGGCTAGCATTACTCGGAATTATTGGGCGTAAAGCGTGCGTAGGCTGCTTAGCAAGTCAGAAGTGAAATCCTTGGGCTCAACCCAAGAACTGCTTTTGAAACTGCTAAGCTTGAGTACTGTAGAGGATAGCGGAATTCCTAGTGTAGAGGTGAAATTCGTAGATATTAGGAGGAACACCGGAGGCGAAGGCGGTTATCTGGGCAGTCACTGACGCTGTTGCACGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCA
GTAAACGATGAGTGCTAGATGTCGGGACTTAGGTTTCGGTGTCGCAGCTAACGCATTAAGCACTCCGCCTGGGGAGTACGACCGCAAGGTTAAAACTCAAAGGAATTGACGGGGACCCGCACAAGCGGTGGAACATGTGGTTTAATTCGATGCTACGCGAAAAACCTTACCAGGCCTTGACATGGTGATCGTATGACTGTGAAAGCAGTTAGTCGGTTCGGCCGGATCATCACAGGTGTTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTCATCCTTAGTTGCCAGCGAGTAATGTCGGGAACTTTAAGGAAACTGCCGGTGATAAGCCGGAAGAAGGTGGGGATGACGTCCAGTCAGCATGGCCCTTACGGCCTGGGCTACACACGTGTTA


In [0]:

#all sequence acquaculture

qiime tools export rep-seqs_aquaculture_big_v3v4.qza --output-path ref_seq_fasta_aquaculture_big_v3v4/

#create rep-seqs_aquaculture_big_v3v4.fa 

mkdir blast_midi

makeblastdb -in rep-seqs_aquaculture_big_v3v4.fa  -dbtype nucl -out blast_midi
#Adding sequences from FASTA; added 24143 sequences in 0.532795 seconds.

blastn -db db/blast_midi -query MLO_assemblata_1008pb.fasta > results_midi.txt
